In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, y_pred, y_true):
        # Aplanar los tensores
        y_pred = y_pred.view(-1)
        y_true = y_true.view(-1)

        intersection = (y_pred * y_true).sum()
        dice = (2. * intersection + self.smooth) / (y_pred.sum() + y_true.sum() + self.smooth)
        return 1 - dice  # Queremos minimizar la pérdida, por lo que devolvemos 1 - Dice

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        """
        Inicializa el Focal Loss.
        
        Parámetros:
        - alpha: Factor de ajuste para el peso de los ejemplos difíciles.
        - gamma: Factor de ajuste para reducir la contribución de ejemplos fáciles.
        - reduction: Tipo de reducción a aplicar ('mean', 'sum' o 'none').
        """
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        """
        Calcula la pérdida focal.

        Parámetros:
        - inputs: Predicciones del modelo (logits).
        - targets: Valores verdaderos (0 o 1 en segmentación binaria).
        """
        # Asegúrate de que los inputs son probabilidades (aplica sigmoide si es necesario)
        inputs = inputs.sigmoid()
        
        # Calcular el Focal Loss
        bce_loss = F.binary_cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-bce_loss)  # pt = exp(-BCE)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * bce_loss

        # Aplicar reducción (mean, sum, o none)
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [ ]:
class BCEDiceLoss(nn.Module):
    """
    Combina BCE (con logits) y Dice Loss:
        L = λ · BCE + (1-λ) · Dice
    donde λ ∈ [0,1].
    """
    def __init__(self, bce_weight: float = 0.5, smooth: float = 1e-6):
        """
        Args
        ----
        bce_weight : proporción de la BCE dentro de la pérdida total.
                     (1-bce_weight) será la proporción de la Dice Loss.
        smooth     : término de suavizado para evitar división por cero en Dice.
        """
        super().__init__()
        self.bce_weight = bce_weight
        self.smooth = smooth
        # Para mayor estabilidad numérica usa logits y BCEWithLogitsLoss
        self.bce = nn.BCEWithLogitsLoss()

    def forward(self, y_pred, y_true):
        """
        y_pred : (N, 1, H, W) logits   ─ sin pasar por sigmoid
        y_true : (N, 1, H, W) etiquetas ─ valores {0,1} o flotantes en [0,1]
        """
        # --- 1) BCE ---
        bce_loss = self.bce(y_pred, y_true)

        # --- 2) Dice ---
        # Aplicar sigmoid para calcular la intersección en el espacio [0,1]
        y_prob = torch.sigmoid(y_pred)
        # Aplanar
        y_prob = y_prob.view(-1)
        y_true = y_true.view(-1)

        intersection = (y_prob * y_true).sum()
        dice_score = (2 * intersection + self.smooth) / (
            y_prob.sum() + y_true.sum() + self.smooth
        )
        dice_loss = 1 - dice_score

        # --- 3) Combinación ---
        loss = self.bce_weight * bce_loss + (1 - self.bce_weight) * dice_loss
        return loss